In [1]:
import requests
import pandas as pd

BASE_URL = "https://www.cheapshark.com/api/1.0/deals"

def fetch_all_deals(store_id="1", page_size=60, max_pages=30):
    """
    Fetch multiple pages of CheapShark deals.

    - store_id="1" means Steam
    - page_size=60 is the maximum number of deals per page
    - max_pages is a safety limit so we don't loop forever
    """
    all_deals = []
    page = 0

    while page < max_pages:
        params = {
            "storeID": store_id,
            "pageSize": page_size,
            "pageNumber": page,
            "sortBy": "Savings",  # sort by biggest discount first
        }
        print(f"Requesting page {page} ...")
        resp = requests.get(BASE_URL, params=params, timeout=30)

        if resp.status_code != 200:
            print(f"  ❌ Status code {resp.status_code}, stopping.")
            break

        deals = resp.json()
        print(f"  Got {len(deals)} deals.")

        if not deals:
            # No more data, we've reached the end
            break

        all_deals.extend(deals)

        # If we got fewer than page_size deals, this is probably the last page
        if len(deals) < page_size:
            break

        page += 1

    print(f"\nTotal deals fetched: {len(all_deals)}")

    df = pd.DataFrame(all_deals)

    # Convert CheapShark's releaseDate (Unix seconds) to datetime, year, and a readable date string
    if "releaseDate" in df.columns:
        df["release_date"] = pd.to_datetime(
            df["releaseDate"],
            unit="s",
            utc=True,
            errors="coerce"
        )
        df["release_year"] = df["release_date"].dt.year

        # 🔹 New column: formatted date string (e.g. 01/01/70, 10/04/16, etc.)
        df["date"] = df["release_date"].dt.strftime("%m/%d/%y")

    return df


df_raw = fetch_all_deals()
df_raw.head()


Requesting page 0 ...


  Got 60 deals.
Requesting page 1 ...


  Got 60 deals.
Requesting page 2 ...


  Got 60 deals.
Requesting page 3 ...


  Got 60 deals.
Requesting page 4 ...


...

  Got 60 deals.
Requesting page 27 ...


  Got 60 deals.
Requesting page 28 ...


  Got 60 deals.
Requesting page 29 ...


  Got 60 deals.

Total deals fetched: 1800


,internalName,title,metacriticLink,dealID,storeID,gameID,salePrice,normalPrice,isOnSale,savings,...,steamRatingPercent,steamRatingCount,steamAppID,releaseDate,lastChange,dealRating,thumb,release_date,release_year,date
0,CUBEDODGE,Cube Dodge,/game/cube-dodge/,5dmWASIZ1%2Bwvsd9F3NMUWt1apOlmuO5Z7Ef%2FF1MOvj...,1,314187,0.79,15.99,1,95.059412,...,0,0,2182710,1667260800,1770414706,2.9,https://shared.fastly.steamstatic.com/store_it...,2022-11-01 00:00:00+00:00,2022,11/01/22
1,STARHIVETHEMENACE,Star Hive: The Menace,/game/star-hive-the-menace/,%2FKD5o2nfM5BN3KtB%2FaWd8yLX972Weo033FXqE6pHJJ...,1,314188,0.79,15.99,1,95.059412,...,0,0,2260310,1677715200,1770414706,2.9,https://shared.fastly.steamstatic.com/store_it...,2023-03-02 00:00:00+00:00,2023,03/02/23
2,EXTREMEOVERTAKE,Extreme Overtake,/game/extreme-overtake/,rE8me7jKWOD%2BlQMdVZ6znY%2BZuvAdwmcJr3ZDeWImL4...,1,314890,0.79,15.99,1,95.059412,...,0,0,3138100,1733529600,1769470525,1.8,https://shared.fastly.steamstatic.com/store_it...,2024-12-07 00:00:00+00:00,2024,12/07/24
3,JUSTMOVEFALLDUNGEONENDLESSABYSS,Just Move Fall Dungeon Endless Abyss,NaN,MUcy6mJcRWSyRrVU6MGNcJAlsUpDljZNJMVL2vRQXV0%3D,1,315908,0.79,15.99,1,95.059412,...,0,0,3856040,0,1770509356,3.1,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970,01/01/70
4,UNCLAIMEDWORLD,Unclaimed World,/game/unclaimed-world/,oOIeSWUAU6bwhCt8eVnsnELwb%2FXff3yYYMvmf%2FezlB...,1,107953,0.84,16.99,1,95.055915,...,49,177,284100,1475539200,1770002642,6.3,https://shared.fastly.steamstatic.com/store_it...,2016-10-04 00:00:00+00:00,2016,10/04/16


In [2]:
df_raw.to_csv("cheapshark_updated.csv", index=False)


In [3]:
# Create a smaller table for the range/arrow plot: top 20 deals by dealRating
df_top20 = (
    df_raw.copy()
    .assign(dealRating_float=pd.to_numeric(df_raw["dealRating"], errors="coerce"))
    .sort_values("dealRating_float", ascending=False)
    .head(20)
)

# Keep only the columns needed for Datawrapper
df_top20_for_chart = df_top20[["title", "normalPrice", "salePrice", "release_year"]]

df_top20_for_chart.head()


,title,normalPrice,salePrice,release_year
92,Super Meat Boy,14.99,1.49,2010
1029,Tomba! Special Edition,19.99,4.99,2025
76,Stories Untold,9.99,0.99,2017
252,The Witcher 2: Assassins of Kings Enhanced Edi...,19.99,2.99,2011
277,Unity of Command II,29.99,4.49,2019


In [4]:
df_top20_for_chart.to_csv("steam_deals_top20_rangeplot.csv", index=False)
